<a href="https://colab.research.google.com/github/jeffvestal/elastic_jupyter_notebooks/blob/main/load_ner_model_from_hf_to_elastic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading a NER Model from Hugging Face into Elasticsearch

This code will show you how to load a Named Entity Recognition (NER) model into elasticsearch.

First we will configure our python environment

1. Set up our python environment
2. Load the chosed model from HF into Elasticsearch
3. Deploy and start the model on ML nodes
4. Test using the NER model


### Elastic version support
Requires Elastic version 8.0+ with a platinum or enterprise license (or trial license)

You can set up a [free trial elasticsearch Deployment in Elastic Cloud](https://cloud.elastic.co/registration).

---
---
# Setup
---
---
This section will set up the python environment with the required libraries

## Install and import required python libraries

Elastic uses the [eland python library](https://github.com/elastic/eland) to download modesl from Hugging Face hub and load them into elasticsearch

In [ ]:
pip install eland

In [ ]:
pip install elasticsearch

In [ ]:
pip install transformers

In [ ]:
pip install sentence_transformers

In [ ]:
pip install torch==1.11

In [ ]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

## Configure elasticsearch authentication. 
The recommended authentication approach is using the [Elastic Cloud ID](https://www.elastic.co/guide/en/cloud/current/ec-cloud-id.html) and a [cluster level API key](https://www.elastic.co/guide/en/kibana/current/api-keys.html)

You can use any method you wish to set the required credentials. We are using getpass in this example to prompt for credentials to avoide storing them in github.

In [7]:
import getpass

In [8]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_api_id = getpass.getpass('Enter cluster API key ID:  ') 
es_api_key = getpass.getpass('Enter cluster API key:  ')

Enter Elastic Cloud ID:  ··········
Enter cluster API key ID:  ··········
Enter cluster API key:  ··········


## Connect to Elastic Cloud

In [ ]:
es = Elasticsearch(cloud_id=es_cloud_id, 
                   api_key=(es_api_id, es_api_key)
                   )
es.info() # should return cluster info

---
---
# Loading a model from Hugging Face model hub
---
---

The Elastic Stack machine learning features support transformer models that conform to the standard BERT model interface and use the WordPiece tokenization algorithm.

A current list of supported architectures can be viewed in the
[Elastic NLP Model Support Docs](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-model-ref.html)


## Download NER model from Hugging Face

For this example we will be using the [dslim/bert-base-NER-uncased](https://huggingface.co/dslim/bert-base-NER-uncased) model


### Download the Model
Here we specify the model id from Hugging Face. The easiest way to get this id is clicking the copy the model name icon next to the name on the model page. 

When calling `TransformerModel` you specify the HF model id and the task type. You can try specifying `auto` and eland will attempt to determine the correct type from info in the model config. This is not always possible so a list of specific `task_type` values can be viewed in the following code: 
[Supported values](https://github.com/elastic/eland/blob/15a300728876022b206161d71055c67b500a0192/eland/ml/pytorch/transformers.py#*L41*)

In [15]:
hf_model_id='dslim/bert-base-NER-uncased'
tm = TransformerModel(hf_model_id, "ner") # 'ner'

### Set and confirm the model ID
To make the name compatible with elasticsearch, the '/' is replaced with '__'



In [16]:
es_model_id = tm.elasticsearch_model_id()
es_model_id

'dslim__bert-base-ner-uncased'

## Export the model in a TorchScrpt representation which Elasticsearch uses

In [17]:
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

## Load the model into Elasticsearch
Model should not already exist in elasticsearch

In [18]:
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config) 

  0%|          | 0/104 [00:00<?, ? parts/s]

---
---
# Starting the Model
---
---

## View information about the model
This is not required but can be handy to get a model overivew

In [19]:
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

{'count': 1,
 'trained_model_configs': [{'model_id': 'dslim__bert-base-ner-uncased',
   'model_type': 'pytorch',
   'created_by': 'api_user',
   'version': '8.6.0',
   'create_time': 1675887938334,
   'model_size_bytes': 0,
   'estimated_operations': 0,
   'license_level': 'platinum',
   'description': "Model dslim/bert-base-NER-uncased for task type 'ner'",
   'tags': [],
   'input': {'field_names': ['text_field']},
   'inference_config': {'ner': {'vocabulary': {'index': '.ml-inference-native-000001'},
     'tokenization': {'bert': {'do_lower_case': True,
       'with_special_tokens': True,
       'max_sequence_length': 512,
       'truncate': 'first',
       'span': -1}},
     'classification_labels': ['O',
      'B_MISC',
      'I_MISC',
      'B_PER',
      'I_PER',
      'B_ORG',
      'I_ORG',
      'B_LOC',
      'I_LOC']}},
   'location': {'index': {'name': '.ml-inference-native-000001'}}}]}

## Deploy the model
This will load the model on the ML nodes and start the process(es) making it available for the NLP task

In [20]:
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body

{'assignment': {'task_parameters': {'model_id': 'dslim__bert-base-ner-uncased',
   'model_bytes': 435663717,
   'threads_per_allocation': 1,
   'number_of_allocations': 1,
   'queue_capacity': 1024,
   'cache_size': '435663717b',
   'priority': 'normal'},
  'routing_table': {'dDqmmPySSKuH8d7HcF54uA': {'current_allocations': 1,
    'target_allocations': 1,
    'routing_state': 'started',
    'reason': ''}},
  'assignment_state': 'started',
  'start_time': '2023-02-08T20:28:50.914490234Z',
  'max_assigned_allocations': 1}}

## Verify the model started without issue

In [21]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']

{'routing_state': 'started'}

---
---
# Generating Entities
---
---
There are two main ways NER modles, and the entities they generate, are used in elasticsearch

1. Calling the `_infer` endpoint with an input string and getting the entities back in the response. The entities are not stored in elastic (directly) and can be used for tasks such as enhancing follow up queries.
2. Passing a field from a document on ingest to a model with an ingest pipeline. The resulting entites are stored along with the document when it is indexed in elasticsearch.

For this example, we will be taking the first approach calling `_infer` to test the NER model works with our data.

# Setting up the index in elasticsearch to store vectors



## Generate Vector for Query

Before we can run a kNN query, we need to convert our query string to a vector.

Creating a sample string

In [22]:
docs =  [
    {
      "text_field": "Hi, my name is Jeff Vestal. I live in Chicago and I work at Elastic"
    }
  ]

We call the `_infer` endpoint supplying the model_id and the doc[s] we want generate entities for. 

In [24]:
response = MlClient.infer_trained_model(es, model_id=es_model_id, docs=docs, )

The vector for the first doc can be accessed in the response dict as shown below

In [26]:
doc_0_vector = response['inference_results']
doc_0_vector

[{'predicted_value': 'Hi, my name is [Jeff Vestal](PER&Jeff+Vestal). I live in [Chicago](LOC&Chicago) and I work at [Elastic](ORG&Elastic)',
  'entities': [{'entity': 'jeff vestal',
    'class_name': 'PER',
    'class_probability': 0.9976065299030981,
    'start_pos': 15,
    'end_pos': 26},
   {'entity': 'chicago',
    'class_name': 'LOC',
    'class_probability': 0.9977499555665439,
    'start_pos': 38,
    'end_pos': 45},
   {'entity': 'elastic',
    'class_name': 'ORG',
    'class_probability': 0.9957462750425009,
    'start_pos': 60,
    'end_pos': 67}]}]